### This code is a Google - Isolated Sign Language Recognition 4th place inferrence code.   

+ Reads pre-trained models and creates tflite.
+ Note: The model in the final submission is an ensemble of six. 
+ However, due to many cases of timeouts, this code restricted to 5 models.



In [1]:
!pip install tflite-runtime
!pip install onnx_tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 8.0 MB/s eta 0:00:00


# 

In [2]:
import os
import json
import tensorflow as tf
import onnx_tf
import onnx
import tflite_runtime
import tflite_runtime.interpreter

from torchvision.ops import StochasticDepth
import math
from tqdm import tqdm
import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn

import warnings
warnings.filterwarnings(action='ignore')


In [3]:

class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.5, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = torch.nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, x, label=None):
        cosine = torch.nn.functional.linear(torch.nn.functional.normalize(
            x), torch.nn.functional.normalize(self.weight)).float()
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        if label is None:
            output = cosine
        else:
            one_hot = torch.zeros(cosine.size(), device='cuda')
            one_hot.scatter_(1, label.cuda().view(-1, 1).long(), 1)
            # you can use torch.where if your torch.__version__ is 0.4
            output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output


class LRUnit1D(nn.Module):
    def __init__(self, in_dim, stochastic_depth_prob=0.0, actf=torch.nn.ReLU()):
        super(LRUnit1D, self).__init__()
        self.layer0 = nn.Linear(in_dim, in_dim, bias=False)
        self.bn0 = nn.BatchNorm1d(in_dim)
        self.layer1 = nn.Linear(in_dim, in_dim, bias=False)
        self.bn1 = nn.BatchNorm1d(in_dim)
        self.actf = actf
        self.stochastic_depth = StochasticDepth(stochastic_depth_prob, "row")

    def forward(self, x):
        h = self.actf(self.bn0(self.layer0(x)))
        h = self.bn1(self.layer1(h))
        h = self.stochastic_depth(h)
        return x + h


class RSUnit1D(nn.Module):
    def __init__(self, in_dim, kernel_size=3, padding=1,
                 padding_mode='zeros', actf=torch.nn.ReLU()):
        super(RSUnit1D, self).__init__()
        self.layer0 = nn.Conv1d(in_dim, in_dim, kernel_size=kernel_size, padding=padding, padding_mode=padding_mode, bias=False)
        self.bn0 = nn.BatchNorm1d(in_dim)
        self.layer1 = nn.Conv1d(in_dim, in_dim, kernel_size=kernel_size, padding=padding, padding_mode=padding_mode, bias=False)
        self.bn1 = nn.BatchNorm1d(in_dim)
        self.actf = actf

    def forward(self, x):
        h = self.actf(self.bn0(self.layer0(x)))
        h = self.bn1(self.layer1(h))
        return x + h



class BackboneVariableLen(nn.Module):
    def __init__(self,
                 in_dim=42,
                 dim1=128,
                 dim2=512,
                 kernel_size=3, padding=1,
                 negative_slope=0.1,
                 ):
        super(BackboneVariableLen, self).__init__()
        self.actf = torch.nn.LeakyReLU(negative_slope=negative_slope)
        self.bn0 = nn.BatchNorm1d(in_dim)
        self.conv1 = nn.Conv1d(in_dim, dim1, kernel_size=kernel_size, padding=padding, bias=True)
        self.conv2 = RSUnit1D(dim1, kernel_size=kernel_size, padding=padding, actf=self.actf)
        self.conv3 = RSUnit1D(dim1, kernel_size=kernel_size, padding=padding, actf=self.actf)
        self.conv4 = RSUnit1D(dim1, kernel_size=1, padding=0, actf=self.actf)
        self.conv5 = RSUnit1D(dim1, kernel_size=1, padding=0, actf=self.actf)
        self.conv6 = RSUnit1D(dim1, kernel_size=1, padding=0, actf=self.actf)
        self.conv6a = nn.Conv1d(dim1, dim2, kernel_size=1, padding=0, bias=True)

        self.flatten = torch.nn.Flatten(1)
        self.g_pool = torch.nn.AdaptiveMaxPool1d(1, return_indices=False)
        self.pool = torch.nn.MaxPool1d(2)

    def forward(self, h):
        h = self.bn0(h)
        h = self.conv1(h)
        h = self.conv2(h)
        h = self.conv3(h)
        h = self.conv4(h)
        h = self.conv5(h)
        h = self.conv6(h)
        h = self.conv6a(h)
        print(h.shape)

        # Remove padding to match during trainning
        h = h[:, :, :-5]

        print(h.shape)
        h = self.g_pool(h)
        h = self.flatten(h)
        return h


class ModelVariableLen(nn.Module):
    def __init__(self,
                 in_dim_H=42,
                 in_dim_L=80,
                 dim1_H=128,
                 dim1_L=64,
                 dim2=512,
                 n_class=250,
                 kernel_size=3, padding=1,
                 arc_m=0.5,
                 arc_s=15,
                 easy_margin=False,
                 stochastic_depth_prob=0.0,
                 negative_slope=0.1,
                 n_recyile=1
                 ):
        super(ModelVariableLen, self).__init__()
        self.actf = torch.nn.LeakyReLU(negative_slope=negative_slope)
        self.hand_module = BackboneVariableLen(in_dim=in_dim_H, dim1=dim1_H, dim2=dim2, negative_slope=negative_slope)
        self.other_module = BackboneVariableLen(in_dim=in_dim_L, dim1=dim1_L, dim2=dim2, negative_slope=negative_slope)

        self.line1 = LRUnit1D(dim2, stochastic_depth_prob, actf=self.actf)
        self.line2 = LRUnit1D(dim2, stochastic_depth_prob, actf=self.actf)
        self.line3 = LRUnit1D(dim2, stochastic_depth_prob, actf=self.actf)
        self.end = ArcMarginProduct(dim2, n_class, easy_margin=easy_margin, s=arc_s, m=arc_m)
        self.flatten = torch.nn.Flatten(1)
        self.g_pool = torch.nn.AdaptiveMaxPool1d(1, return_indices=False)
        self.pool = torch.nn.MaxPool1d(2)

        self.n_recyile = n_recyile

    def forward(self, hand, other):
        hand = self.hand_module(hand)
        other = self.other_module(other)
        h = hand + other
        for n in range(self.n_recyile):
            h = self.line1(h)
        for n in range(self.n_recyile):
            h = self.line2(h)
        for n in range(self.n_recyile):
            h = self.line3(h)
        h = self.end(h)
        return h

# %%


class BackboneFixedLen(nn.Module):
    def __init__(self,
                 in_dim=42,
                 dim1=128,
                 dim2=512,
                 kernel_size=3, padding=1,
                 negative_slope=0.1,
                 ):
        super(BackboneFixedLen, self).__init__()
        self.actf = torch.nn.LeakyReLU(negative_slope=negative_slope)
        self.bn0 = nn.BatchNorm1d(in_dim)
        self.conv1 = nn.Conv1d(in_dim, dim1, kernel_size=3, padding=1, bias=True)
        self.conv2 = RSUnit1D(dim1, kernel_size=kernel_size, padding=padding, actf=self.actf)
        self.conv3 = RSUnit1D(dim1, kernel_size=kernel_size, padding=padding, actf=self.actf)
        self.conv4 = RSUnit1D(dim1, kernel_size=kernel_size, padding=padding, actf=self.actf)
        self.conv5 = RSUnit1D(dim1, kernel_size=kernel_size, padding=padding, actf=self.actf)
        self.conv6 = RSUnit1D(dim1, kernel_size=kernel_size, padding=padding, actf=self.actf)
        self.conv6a = nn.Conv1d(dim1, dim2, kernel_size=1, padding=0, bias=True)

        self.flatten = torch.nn.Flatten(1)
        self.g_pool = torch.nn.AdaptiveMaxPool1d(1, return_indices=False)
        self.pool = torch.nn.MaxPool1d(2)

    def forward(self, x):
        h = self.bn0(x)
        h = self.conv1(h)
        h = self.conv2(h)
        h = self.pool(h)
        h = self.conv3(h)
        h = self.pool(h)
        h = self.conv4(h)
        h = self.pool(h)
        h = self.conv5(h)
        h = self.conv6(h)
        h = self.conv6a(h)
        h = self.g_pool(h)
        h = self.flatten(h)
        return h


class ModelFixedLen(nn.Module):
    def __init__(self,
                 in_dim_H=42,
                 in_dim_L=80,
                 dim1_H=128,
                 dim1_L=64,
                 dim2=512,
                 n_class=250,
                 kernel_size=3, padding=1,
                 arc_m=0.5,
                 arc_s=15,
                 easy_margin=False,
                 stochastic_depth_prob=0.0,
                 negative_slope=0.1,
                 n_recyile=1
                 ):
        super(ModelFixedLen, self).__init__()
        self.actf = torch.nn.LeakyReLU(negative_slope=negative_slope)
        self.hand_module = BackboneFixedLen(in_dim=in_dim_H, dim1=dim1_H, dim2=dim2, negative_slope=negative_slope)
        self.other_module = BackboneFixedLen(in_dim=in_dim_L, dim1=dim1_L, dim2=dim2, negative_slope=negative_slope)

        self.line1 = LRUnit1D(dim2, stochastic_depth_prob, actf=self.actf)
        self.line2 = LRUnit1D(dim2, stochastic_depth_prob, actf=self.actf)
        self.line3 = LRUnit1D(dim2, stochastic_depth_prob, actf=self.actf)
        self.end = ArcMarginProduct(dim2, n_class, easy_margin=easy_margin, s=arc_s, m=arc_m)
        self.flatten = torch.nn.Flatten(1)
        self.g_pool = torch.nn.AdaptiveMaxPool1d(1, return_indices=False)
        self.pool = torch.nn.MaxPool1d(2)

        self.n_recyile = n_recyile

    def forward(self, hand, other):
        hand = self.hand_module(hand)
        other = self.other_module(other)
        h = hand + other
        for n in range(self.n_recyile):
            h = self.line1(h)
        for n in range(self.n_recyile):
            h = self.line2(h)
        for n in range(self.n_recyile):
            h = self.line3(h)
        h = self.end(h)
        return h


In [4]:

@torch.jit.script
def normalize_by_midwayBetweenEyes(x):

    midwayBetweenEyes = x[:, 168]
    mask = ~torch.isnan(midwayBetweenEyes[:, 0])
    masked = midwayBetweenEyes[mask]  # .mean(keepdims=True)
    m = torch.mean(masked, dim=0)
    if torch.any(torch.isnan(m)): 
        return x
    else:
        m = torch.unsqueeze(m, 0)
        m = torch.unsqueeze(m, 0)
    return x - m


@torch.jit.script
def apply_hand_mask(x):

    hand_mask = ~torch.isnan(x[:, 0, 0])
    m = x[~torch.isnan(x)].mean(0)  # noramlisation to common maen
    m = torch.unsqueeze(m, 0)
    m = torch.unsqueeze(m, 0)
    x = x - m
    s = x[~torch.isnan(x)].std(0)
    s = torch.unsqueeze(s, 0)
    s = torch.unsqueeze(s, 0)
    x = x / s
    x = torch.where(torch.isnan(x), torch.tensor(0.0, dtype=torch.float32), x)

    if hand_mask.sum() > 0:
        hand = x[hand_mask, :21]
    else:
        hand = x[:, :21]
    other = x[:, 21:]
    return hand, other


@torch.jit.script
def apply_autoflip(xy):

    left_start_index = 468
    right_start_index = 522
    length = xy[:, 0, 0].shape[0]
    l_num = length - torch.isnan(xy[:, left_start_index, 0]).sum()
    r_num = length - torch.isnan(xy[:, right_start_index, 0]).sum()

    # 左手主体の場合
    if r_num < l_num:
        xy = torch.stack((-xy[:, :, 0], xy[:, :, 1]), dim=2)

        hand_indexes = torch.tensor((468, 469, 470, 471, 472, 473, 474, 475, 476, 477,
                                     478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488))

        lipsUpperOuter_indexes = torch.tensor((291, 409, 270, 269, 267, 0, 37, 39, 40, 185, 61))
        lipsLowerOuter_indexes = torch.tensor((375, 321, 405, 314, 17, 84, 181, 91, 46))
        lipsUpperInner_indexes = torch.tensor((308, 415, 310, 311, 312, 13, 82, 81, 80, 191, 78))
        lipsLowerInner_indexes = torch.tensor((324, 318, 402, 317, 14, 87, 178, 88, 95))

    else:

        hand_indexes = torch.tensor((522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534,
                                     535, 536, 537, 538, 539, 540, 541, 542))

        lipsUpperOuter_indexes = torch.tensor((61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291))
        lipsLowerOuter_indexes = torch.tensor((46, 91, 181, 84, 17, 314, 405, 321, 375))
        lipsUpperInner_indexes = torch.tensor((78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308))
        lipsLowerInner_indexes = torch.tensor((95, 88, 178, 87, 14, 317, 402, 318, 324))

    # l_hand = xy[:, l_hand_indexes]
    hand = xy[:, hand_indexes]

    lipsUpperOuter = xy[:, lipsUpperOuter_indexes]
    lipsLowerOuter = xy[:, lipsLowerOuter_indexes]
    lipsUpperInner = xy[:, lipsUpperInner_indexes]
    lipsLowerInner = xy[:, lipsLowerInner_indexes]

    x = torch.concat((hand,
                      lipsUpperOuter,
                      lipsLowerOuter,
                      lipsUpperInner,
                      lipsLowerInner), dim=1)
    print("feature", x.shape)
    return x


class AslData_tf(nn.Module):

    def __init__(self, seq_length):
        super(AslData_tf, self).__init__()
        self.seq_length = seq_length

    def forward(self, data):
        # TxDx3
        # print(x.shape)

        xy = data[:, :, :2]  # TxDx2
        xy = normalize_by_midwayBetweenEyes(xy)

        # x = x[:, self.indexes]  # Txdx2
        xy = apply_autoflip(xy)

        hand, other = apply_hand_mask(xy)

        hand = torch.reshape(hand, (hand.shape[0], -1))  # T , 2*d
        hand = torch.permute(hand, (1, 0))  # 2*d, T
        print(hand.shape)
        other = torch.reshape(other, (other.shape[0], -1))  # T , 2*d
        other = torch.permute(other, (1, 0))  # 2*d, T
        print(other.shape)

        hand = torch.unsqueeze(hand, 0)
        other = torch.unsqueeze(other, 0)

        hand_fixed_len = torch.unsqueeze(hand, 0)
        hand_fixed_len = torch.nn.functional.interpolate(hand_fixed_len, size=(hand_fixed_len.shape[2], self.seq_length))
        hand_fixed_len = hand_fixed_len[0]

        other_fixed_len = torch.unsqueeze(other, 0)
        other_fixed_len = torch.nn.functional.interpolate(other_fixed_len, size=(other_fixed_len.shape[2], self.seq_length))
        other_fixed_len = other_fixed_len[0]

        # Since the batch size is 1 during inference,
        # there is no need for padding, but only 5 padding is used to match during learning.
        hand_variable_len = torch.nn.ConstantPad1d((0, 5), 0)(hand)
        other_variable_len = torch.nn.ConstantPad1d((0, 5), 0)(other)

        print("hand_fixed_len", hand_fixed_len.shape)
        print("other_fixed_len", other_fixed_len.shape)
        print("hand_variable_len", hand_variable_len.shape)
        print("other_variable_len", other_variable_len.shape)
        return hand_fixed_len, other_fixed_len, hand_variable_len, other_variable_len


In [5]:

class SubmitModel(torch.nn.Module):
    def __init__(self,
                 ):
        
        super(SubmitModel, self).__init__()
        self.prepare = AslData_tf(SEQ_LEN)
        self.fix_len_models = nn.ModuleList()
        self.var_len_models = nn.ModuleList()

    def __call__(self, x):
        
        hand_fixed_len, other_fixed_len, hand_variable_len, other_variable_len = self.prepare(x)
        print(len(self.fix_len_models), len(self.var_len_models))

        ys = []
        for i in range(len(self.fix_len_models)):
            ys.append((self.fix_len_models[i](hand_fixed_len, other_fixed_len)))
        for i in range(len(self.var_len_models)):
            ys.append((self.var_len_models[i](hand_variable_len, other_variable_len)))
        y = torch.cat(ys, dim=0).mean(dim=0)
        return y



In [6]:
def load_relevant_data_subset(pq_path):
    #定义三个列，xyz维度
    data_columns = ['x', 'y', 'z']
    #从给定路径，指定列读出df
    data = pd.read_parquet(pq_path, columns=data_columns)
    #计算数据中有多少个帧
    n_frames = int(len(data) / ROWS_PER_FRAME)
    #转换矩阵行数和列数，变成n_frames页，ROWS_PER_FRAME行，3列的三维矩阵
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    #把data里面的数据都转换成np.float32格式
    return data.astype(np.float32)

DATA_PATH = "../data/"
INPUT_PATH = "../input/asl-signs/"
LANDMARK_FILES_DIR = os.path.join(INPUT_PATH, "train_landmark_files")
TRAIN_FILE = os.path.join(INPUT_PATH, "train.csv")
JSON_FILE = os.path.join(INPUT_PATH, "sign_to_prediction_index_map.json")

n_class = 250
#每一帧的像素行数？关节点数
ROWS_PER_FRAME = 543

SEQ_LEN = 96
arc_s = 47
arc_m = 0.0
easy_margin = True
negative_slope = 0.1

in_dim_H = 42
in_dim_L = 80
dim1_H = 128
dim1_L = 64
dim2 = 512
opset_version=12

result_path = "/kaggle/input/asl-4th-place-model/ASL_4th_place_model/"


In [7]:
model = SubmitModel()

model_path = os.path.join(result_path, "ModelFixedLen_deep_seed0_cleanlab_False", "swa_300.pt")
tmp_model = ModelFixedLen(in_dim_H=in_dim_H,
                          in_dim_L=in_dim_L,
                          dim1_H=dim1_H,
                          dim1_L=dim1_L,
                          dim2=dim2,
                          easy_margin=easy_margin,
                          arc_s=arc_s, arc_m=arc_m,
                          stochastic_depth_prob=0.5,
                          negative_slope=negative_slope,
                          n_recyile=3
                          )
swa_model = torch.optim.swa_utils.AveragedModel(tmp_model)
swa_model.module.eval()
swa_model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.fix_len_models.append(swa_model.module)
"""
model_path = os.path.join(result_path, "ModelFixedLen_deep_seed5_cleanlab_True", "swa_300.pt")
tmp_model = ModelFixedLen(in_dim_H=in_dim_H,
                          in_dim_L=in_dim_L,
                          dim1_H=dim1_H,
                          dim1_L=dim1_L,
                          dim2=dim2,
                          easy_margin=easy_margin,
                          arc_s=arc_s, arc_m=arc_m,
                          stochastic_depth_prob=0.5,
                          negative_slope=negative_slope,
                          n_recyile=3
                          )
swa_model = torch.optim.swa_utils.AveragedModel(tmp_model)
swa_model.module.eval()
swa_model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.fix_len_models.append(swa_model.module)
"""
# %%
model_path = os.path.join(result_path, "ModelFixedLen_deep_seed6_cleanlab_True", "swa_300.pt")
tmp_model = ModelFixedLen(in_dim_H=in_dim_H,
                          in_dim_L=in_dim_L,
                          dim1_H=dim1_H,
                          dim1_L=dim1_L,
                          dim2=dim2,
                          easy_margin=easy_margin,
                          arc_s=arc_s, arc_m=arc_m,
                          stochastic_depth_prob=0.5,
                          negative_slope=negative_slope,
                          n_recyile=3
                          )
swa_model = torch.optim.swa_utils.AveragedModel(tmp_model)
swa_model.module.eval()
swa_model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.fix_len_models.append(swa_model.module)
# %%
model_path = os.path.join(result_path, "ModelFixedLen_shallow_seed5_cleanlab_False", "swa_300.pt")
tmp_model = ModelFixedLen(in_dim_H=in_dim_H,
                          in_dim_L=in_dim_L,
                          dim1_H=dim1_H,
                          dim1_L=dim1_L,
                          dim2=dim2,
                          easy_margin=easy_margin,
                          arc_s=arc_s, arc_m=arc_m,
                          stochastic_depth_prob=0.1,
                          negative_slope=negative_slope,
                          n_recyile=1
                          )
swa_model = torch.optim.swa_utils.AveragedModel(tmp_model)
swa_model.module.eval()
swa_model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.fix_len_models.append(swa_model.module)

# %%
model_path = os.path.join(result_path, "ModelVariableLen_deep_seed35_cleanlab_True", "swa_300.pt")
tmp_model = ModelVariableLen(in_dim_H=in_dim_H, in_dim_L=in_dim_L,
                             dim1_H=dim1_H,
                             dim1_L=dim1_L,
                             dim2=dim2,
                             easy_margin=easy_margin,
                             arc_s=arc_s, arc_m=arc_m,
                             stochastic_depth_prob=0.5,
                             negative_slope=negative_slope,
                             n_recyile=3
                             )
swa_model = torch.optim.swa_utils.AveragedModel(tmp_model)
swa_model.module.eval()
swa_model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.var_len_models.append(swa_model.module)

# %%
model_path = os.path.join(result_path, "ModelVariableLen_shallow_seed150_cleanlab_False", "swa_300.pt")
tmp_model = ModelVariableLen(in_dim_H=in_dim_H, in_dim_L=in_dim_L,
                             dim1_H=dim1_H,
                             dim1_L=dim1_L,
                             dim2=dim2,
                             easy_margin=easy_margin,
                             arc_s=arc_s, arc_m=arc_m,
                             stochastic_depth_prob=0.1,
                             negative_slope=negative_slope,
                             n_recyile=1
                             )
swa_model = torch.optim.swa_utils.AveragedModel(tmp_model)
swa_model.module.eval()
swa_model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.var_len_models.append(swa_model.module)


ModuleList(
  (0): ModelVariableLen(
    (actf): LeakyReLU(negative_slope=0.1)
    (hand_module): BackboneVariableLen(
      (actf): LeakyReLU(negative_slope=0.1)
      (bn0): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv1d(42, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): RSUnit1D(
        (layer0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (layer1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (actf): LeakyReLU(negative_slope=0.1)
      )
      (conv3): RSUnit1D(
        (layer0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [8]:
#读train.csv存到一个df里，里面有.Parquet文件路径，以及其中数据的标签
df = pd.read_csv(TRAIN_FILE)
#读sign_to_prediction_index_map.json，里面有每个标签对应数字的映射
label_map = json.load(open(JSON_FILE, "r"))
#给df加一行lable，里面存的是每一行的标签对应的数字
df['label'] = df['sign'].map(label_map)
#取出第一行的数据
row = df.loc[0]
#读第一行path里对应的.Parquet文件，转换成一个帧数*每帧像素行数*xyz三列的数组
sample_input = load_relevant_data_subset(os.path.join(INPUT_PATH, row.path))
#把np矩阵转换成Tensor格式的矩阵，更适合深度学习训练
sample_input = torch.tensor(sample_input)
#把处理好的数据放进自定义的model里面
a = model(sample_input)


feature [23, 61, 2]
torch.Size([42, 11])
torch.Size([80, 23])
hand_fixed_len torch.Size([1, 42, 96])
other_fixed_len torch.Size([1, 80, 96])
hand_variable_len torch.Size([1, 42, 16])
other_variable_len torch.Size([1, 80, 28])
3 2
torch.Size([1, 512, 16])
torch.Size([1, 512, 11])
torch.Size([1, 512, 28])
torch.Size([1, 512, 23])
torch.Size([1, 512, 16])
torch.Size([1, 512, 11])
torch.Size([1, 512, 28])
torch.Size([1, 512, 23])


In [9]:
models_path = "/kaggle/working/"

os.makedirs(models_path, exist_ok=True)
onnx_feat_gen_path = os.path.join(models_path, 'model.onnx')
tf_feat_gen_path = os.path.join(models_path, 'tf_model')
tflite_model_path = os.path.join(models_path, 'model.tflite')


print(sample_input.shape)

torch.onnx.export(
    model,                  # PyTorch Model
    sample_input,                    # Input tensor
    onnx_feat_gen_path,        # Output file (eg. 'output_model.onnx')
    opset_version=opset_version,       # Operator support version
    input_names=['inputs'],   # Input tensor name (arbitary)
    output_names=['outputs'],  # Output tensor name (arbitary)
    dynamic_axes={
        'inputs': {0: 'inputs',
                   1: 'length'},
        'outputs': {0: 'inputs'},
    }
)

torch.Size([23, 543, 3])
feature [23, 61, 2]
torch.Size([42, 11])
torch.Size([80, 23])
hand_fixed_len torch.Size([1, 42, 96])
other_fixed_len torch.Size([1, 80, 96])
hand_variable_len torch.Size([1, 42, 16])
other_variable_len torch.Size([1, 80, 28])
3 2
torch.Size([1, 512, 16])
torch.Size([1, 512, 11])
torch.Size([1, 512, 28])
torch.Size([1, 512, 23])
torch.Size([1, 512, 16])
torch.Size([1, 512, 11])
torch.Size([1, 512, 28])
torch.Size([1, 512, 23])


In [10]:
print("converting from onnx to tensorflow...")
onnx_feat_gen = onnx.load(onnx_feat_gen_path)
tf_rep = onnx_tf.backend.prepare(onnx_feat_gen)
tf_rep.export_graph(tf_feat_gen_path)
del tf_rep
# %%
print("converting from tensorflow to tflite")
converter = tf.lite.TFLiteConverter.from_saved_model(tf_feat_gen_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
print("converting finish!!")

converting from onnx to tensorflow...
converting from tensorflow to tflite
converting finish!!


In [11]:
interpreter = tflite_runtime.interpreter.Interpreter(tflite_model_path)
interpreter.allocate_tensors()

found_signatures = list(interpreter.get_signature_list().keys())

#%%
if False:
    prediction_fn = interpreter.get_signature_runner("serving_default")
    ok = []
    for i in tqdm(range(len(df))):
        row = df.loc[i]
        a = load_relevant_data_subset(os.path.join(INPUT_PATH, row.path))
        output = prediction_fn(inputs=a)
        z = output["outputs"]
        sign = np.argmax(z)
        ok.append(row.label == sign)
        if len(ok) % 1000 == 0:
            print(np.mean(ok))
    print(np.mean(ok))
    # %%


In [12]:
!zip submission.zip $tflite_model_path

  adding: kaggle/working/model.tflite (deflated 9%)
